# Data Exploration 

In [1]:
import pandas as pd

df = pd.read_json('data/sample_output_json.json ', lines=True)
# Convert 'date' column to datetime with the correct format
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')


In [2]:
df.date.min(), df.date.max()

(Timestamp('2021-06-04 00:00:00'), Timestamp('2024-03-01 00:00:00'))

In [17]:
df.head(2).to_dict()

{'text': {0: 'Towards Long Context RAG \n\nGemini 1.5 Pro is impressive. Naturally this begs this question of what RAG will look like in a long-context LLM future - which techniques will disappear and which will remain?\n\nWe did a deep dive into Gemini, and consolidated our thinking about…',
  1: 'LayerDiffusion 这个可以直接生成透明背景图片的项目已经可以在forge的扩展上使用了，试了一下确实非常牛批。\n\n不仅支持直接生成透明的图片元素，还支持在已有图片上生成跟环境融合的透明图片，这个就可以做很多事情了。\n\n不得不怀疑老哥只支持forge是为了推广他的项目，哈哈…'},
 'author_name': {0: 'LlamaIndex', 1: '歸藏'},
 'author_handle': {0: '@llama_index', 1: '@op7418'},
 'date': {0: Timestamp('2024-03-01 00:00:00'),
  1: Timestamp('2024-03-01 00:00:00')},
 'lang': {0: 'en', 1: 'zh'},
 'url': {0: 'https://twitter.com/llama_index/status/1763620476847632744',
  1: 'https://twitter.com/op7418/status/1763563619193856252'},
 'mentioned_urls': {0: [], 1: []},
 'is_retweet': {0: False, 1: False},
 'media_type': {0: 'Image', 1: 'Video'},
 'images_urls': {0: ['https://pbs.twimg.com/media/GHmjUsMaEAA5hRQ?format=png&name=900x

In [3]:
df.sample(7)

,text,author_name,author_handle,date,lang,url,mentioned_urls,is_retweet,media_type,images_urls,num_reply,num_retweet,num_like,num_view
660,BEIR is a *zero-shot test* set. It’s not meant...,Omar Khattab,@lateinteraction,2023-07-16,en,https://twitter.com/lateinteraction/status/168...,[],False,No media,None,2,1,33,0
484,,Out Of Context Football,@nocontextfooty,2023-09-25,,https://twitter.com/nocontextfooty/status/1706...,[],False,Video,None,367,6482,85899,0
379,"Here's my conversation with \n@elonmusk\n, his...",Lex Fridman,@lexfridman,2023-11-09,en,https://twitter.com/lexfridman/status/17226860...,[],False,Video,None,3153,6709,34571,0
395,,gaut,@0xgaut,2023-11-03,,https://twitter.com/0xgaut/status/172053180438...,[],False,Image,[https://pbs.twimg.com/media/F-COjmwXAAEHzyB?f...,311,1450,11053,0
138,Azure 的 zh-CN-XiaoxiaoDialectsNeural 这个语音是同一个人...,LE,@lewangx,2024-01-16,zh,https://twitter.com/lewangx/status/17471196773...,[],False,Video,None,9,62,193,0
563,みなとみらいでポケモンドローンのタイムラプス,k_kinukawa,@k_kinukawa,2023-08-11,ja,https://twitter.com/k_kinukawa/status/16899922...,[],False,Video,None,42,19295,88132,0
739,Every science and tech person who is currently...,Yishan,@yishan,2023-06-19,en,https://twitter.com/yishan/status/167061296506...,[],False,No media,None,607,224,1420,0


In [4]:
df.author_name.value_counts().head(5)

author_name
Jim Fan         23
Jerry Liu       20
Yann LeCun      19
Science girl    18
Massimo         17
Name: count, dtype: int64

## Plots
Few basic plots.

In [5]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

def plot_likes_with_cumulative(df, start_date, end_date):
    df['date'] = pd.to_datetime(df['date'])
    df = df[(df['date'] >= start_date) & (df['date'] <= end_date)]
    df['media_type'] = df['media_type'].replace({'No media': 'Text'})
    df['year_month'] = df['date'].dt.strftime('%Y-%m')

    # Prepare the monthly count DataFrame
    monthly_likes_df = df.groupby(['year_month', 'media_type']).size().reset_index(name='counts')

    # Prepare the cumulative DataFrame
    cumulative_likes_df = monthly_likes_df.copy()
    cumulative_likes_df['cumulative_counts'] = cumulative_likes_df.groupby('media_type')['counts'].cumsum()

    # Select a color palette
    color_palette = px.colors.qualitative.Pastel

    # Match colors to media type
    color_map = {media_type: color_palette[i] for i, media_type in enumerate([ 'Video', 'Image',  'Text'])}

    # Creating subplots
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=("Monthly Likes by Media Type", "Cumulative Likes by Media Type"),
                        vertical_spacing=0.15)

    # Adding the monthly bar chart
    for media_type in monthly_likes_df['media_type'].unique():
        filtered_df = monthly_likes_df[monthly_likes_df['media_type'] == media_type]
        fig.add_trace(go.Bar(x=filtered_df['year_month'], y=filtered_df['counts'], opacity=0.8,
                             name=media_type, marker_color=color_map[media_type]),
                      row=1, col=1)

    # Adding the cumulative area chart
    for media_type in cumulative_likes_df['media_type'].unique():
        filtered_cumulative_df = cumulative_likes_df[cumulative_likes_df['media_type'] == media_type]
        fig.add_trace(go.Scatter(x=filtered_cumulative_df['year_month'], y=filtered_cumulative_df['cumulative_counts'],
                                 mode='lines', name=media_type, fill='tonexty',
                                 line=dict(color=color_map[media_type])),
                      row=2, col=1)

    # Update layout for clarity and visual appeal
    fig.update_layout(height=600, title_text="Likes Analysis by Media Type")
    fig.update_xaxes(title_text="Month-Year", row=2, col=1)
    fig.update_yaxes(title_text="Monthly Counts", row=1, col=1)
    fig.update_yaxes(title_text="Cumulative Counts", row=2, col=1)

    fig.show()

# Call the function with your DataFrame and a date range
plot_likes_with_cumulative(df, '2023-01-01', '2023-12-31')


C:\Users\zjxen\AppData\Local\Temp\ipykernel_46600\3397730805.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['media_type'] = df['media_type'].replace({'No media': 'Text'})
C:\Users\zjxen\AppData\Local\Temp\ipykernel_46600\3397730805.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year_month'] = df['date'].dt.strftime('%Y-%m')


In [6]:
def plot_combined_donut_chart_ordered_correctly(data_df, date_col = 'StrTime'):
    # Convert 'StrTime' to datetime and extract the weekday name and number
    data_df['WeekdayName'] = data_df[date_col].dt.day_name()
    # Map weekday names to numbers for sorting (Monday=0, ..., Sunday=6)
    weekdays_map = {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}
    data_df['WeekdayNumber'] = data_df['WeekdayName'].map(weekdays_map)
    
    # Prepare the data: count messages by WeekdayName, ignoring sender
    weekday_distribution = data_df.groupby(['WeekdayName', 'WeekdayNumber']).size().reset_index(name='Count')
    
    # Sort by WeekdayNumber to ensure correct order
    weekday_distribution.sort_values('WeekdayNumber', inplace=True)
    
    # Now that sorting is done, WeekdayNumber is no longer needed
    weekday_distribution = weekday_distribution.drop(columns=['WeekdayNumber'])
    
    # Create Donut chart with a different color scheme
    fig = px.pie(weekday_distribution, names='WeekdayName', values='Count',
                 title="Tweets Liked by Weekday",
                 color_discrete_sequence=px.colors.qualitative.Pastel,
                 hole=0.4)
    
    # Add slight margin between each day for better visualization
    fig.update_traces(textinfo='percent+label', pull=0.02)
    fig.update_traces(
        sort=False, 
        direction='clockwise'
                      )


    fig.show()

# Ensure data_df['StrTime'] is a datetime column before calling the function
plot_combined_donut_chart_ordered_correctly(df, date_col='date')


In [7]:
from plotly_calplot import calplot

def convert_to_day_df(df):
    # Convert the 'date' column to datetime if it's not already
    df['date'] = pd.to_datetime(df['date'])
    day_df = df.groupby(df['date'].dt.date).size().reset_index(name='num_tweets')
    
    return day_df

day_df = convert_to_day_df(df)
day_df.head(2)

fig = calplot(day_df[(day_df['date'].astype(str)>'2023-01-01') & (day_df['date'].astype(str) < '2024-01-01')], x="date", y="num_tweets", 
              start_month = 1,
    end_month = 12,
    )

# set height of the figure
fig.update_layout(height=250)
# add title 
fig.update_layout(title='Number of Liked Tweets per Day 2023')
fig.show()

c:\Users\zjxen\anaconda3\envs\openai\lib\site-packages\plotly_calplot\calplot.py:159: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Image captions 
Proof of concept: get high level summary, detailed description and tags for first image from liked tweets. 


With GPT-4-V. 

Gemini 1.0 also works pretty great for this usecase.

Mainly, convert image to structured json data. 

In [10]:
from openai import OpenAI
from config import OPENAI_API_KEY
import os 
import json
from IPython.display import Image, display

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

client = OpenAI()

def get_valid_json_with_gpt35(input):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-0125", # latest version of GPT-3.5, better JSON mode support.
        messages=[
            {
            "role": "system",
            "content": "You will be given a JSON string (that may not be completely valid), your task is to create a valid JSON results based on the input. Try keep the structure of the input JSON as much as possible."
            }, 
            {
            "role": "user",
            "content": input
            }
        ],
        temperature=0.1,
        response_format = { "type": "json_object" },
        max_tokens=2000,
        )
    return response.choices[0].message.content

def get_tweet_image_captioning(image_url, author_name, text):
    image_prompt = f"""
    This is a tweet from {author_name}, 
    with the following text:
    {text}.

    Below is an image from the tweet.
    Your task is to caption the image and provide a description. reference the text or author if necessary. 

    Output rules you MUST follow:
    - "Summary": A brief high level, big picture description of the image. What is this and what is this for?
    - "Description": A detailed description of the image. If there are text in the image, please include all of it in the description. (in its original language)
    - "Tags": a brief list of high level tags that describe the image. (e.g. "cat", "anime girl", "food")

    Return results in a JSON format (with "Answer" as key, contains list of  dictionary with exactly these key names):    
"""

    vision_response = client.chat.completions.create(
        model="gpt-4-vision-preview",
        messages=[
            {
            "role": "user",
            "content": [
                {"type": "text", "text": image_prompt},
                {
                "type": "image_url",
                "image_url": {
                    "url": image_url,
                },
                },
            ],
            }
        ],
        max_tokens=1000,
        )
    
    # Post process the response with GPT 3.5 json mode
    json_captions = get_valid_json_with_gpt35(vision_response.choices[0].message.content)
    return json_captions


In [35]:
sample_image_df = df[df.media_type=='Image'].head(30)
sample_image_urls = sample_image_df.images_urls.tolist()
sample_authors = sample_image_df.author_name.tolist()
sample_texts = sample_image_df.text.tolist()

In [42]:
cur_idx = 26
image_url = sample_image_urls[cur_idx][0]
author_name = sample_authors[cur_idx]
text = sample_texts[cur_idx]

display(Image(url=image_url))

In [43]:

print(f"Image url: {image_url}\n"
      f"Author name: {author_name}\n"
      f"Text: {text}"   )

result = get_tweet_image_captioning(image_url, author_name, text)

Image url: https://pbs.twimg.com/media/GGQj7ObbQAAaKgl?format=jpg&name=900x900
Author name: Kars
Text: Cover for Fate/Strange Fake volume 9 which will release on March 8.

https://dengekibunko.jp/product/fate/322308000304.html… #strangefake


In [44]:
print(json.dumps(json.loads(result), indent=4))

{
    "Answer": [
        {
            "Summary": "The image is an illustrated cover art for 'Fate/Strange Fake' volume 9 which is set to release on March 8 as mentioned in the tweet by Kars.",
            "Description": "The image features an anime-style illustration of a red-haired female character with a determined expression. She is wearing a white Greek-style toga with a purple sash, which appears to be unraveling, and black and white armored gloves. The character has a red choker and earrings, and blood is visible on her shoulder. Behind her, a chaotic burst illustrates a dramatic scene with dark red visual effects that resemble flames or shards of destruction, contrasting against the brighter colors in the foreground. There is no text within the image itself.",
            "Tags": [
                "anime",
                "book cover",
                "Fate/Strange Fake",
                "illustration",
                "red-haired anime character"
            ]
        }
    ]

## Download images

In [10]:
df.head(2)

,text,author_name,author_handle,date,lang,url,mentioned_urls,is_retweet,media_type,images_urls,num_reply,num_retweet,num_like,num_view,WeekdayName,WeekdayNumber
0,Towards Long Context RAG \n\nGemini 1.5 Pro is...,LlamaIndex,@llama_index,2024-03-01,en,https://twitter.com/llama_index/status/1763620...,[],False,Image,[https://pbs.twimg.com/media/GHmjUsMaEAA5hRQ?f...,3,69,298,0,Friday,4
1,LayerDiffusion 这个可以直接生成透明背景图片的项目已经可以在forge的扩展上...,歸藏,@op7418,2024-03-01,zh,https://twitter.com/op7418/status/176356361919...,[],False,Video,None,7,71,267,0,Friday,4


In [13]:
import os
import requests
import time
from urllib.parse import urlparse

def download_images(df, output_dir='downloaded_images'):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for _, row in df.iterrows():
        if row['images_urls'] is not None:
            url_parts = urlparse(row['url'])
            twitter_name = url_parts.path.split('/')[1]
            tweet_id = url_parts.path.split('/')[-1]

            for i, image_url in enumerate(row['images_urls'], start=1):
                image_id = f"{twitter_name}__{tweet_id}_{i}"
                image_path = os.path.join(output_dir, f"{image_id}.jpg")

                try:
                    response = requests.get(image_url, stream=True)
                    response.raise_for_status()

                    with open(image_path, 'wb') as file:
                        for chunk in response.iter_content(chunk_size=8192):
                            file.write(chunk)

                    print(f"Downloaded image: {image_id}")
                    time.sleep(1)  # Add a delay to avoid being banned by Twitter

                except requests.exceptions.RequestException as e:
                    print(f"Error downloading image: {image_id}")
                    print(f"Error message: {str(e)}")

    print("Image download completed.")

In [15]:
df.shape 

(1026, 16)

In [16]:
download_images(df, output_dir='downloaded_images')

Downloaded image: llama_index__1763620476847632744_1
Downloaded image: MadSealCutter__1763107184580112694_1
Downloaded image: sophiamyang__1762125912051519947_1
Downloaded image: Sentdex__1761574584959095097_1
Downloaded image: OrwellNGoode__1760960242915930329_1
Downloaded image: LostLeanore__1760738125918425471_1
Downloaded image: FedeItaliano76__1760596748764917765_1
Downloaded image: FedeItaliano76__1760596748764917765_2
Downloaded image: FedeItaliano76__1760596748764917765_3
Downloaded image: FedeItaliano76__1760596748764917765_4
Downloaded image: chrisalbon__1760760129300070602_1
Downloaded image: DrJimFan__1760695377651781950_1
Downloaded image: DrJimFan__1760695377651781950_2
Downloaded image: DrJimFan__1760695377651781950_3
Downloaded image: DrJimFan__1760695377651781950_4
Downloaded image: elonmusk__1759295781196927438_1
Downloaded image: elonmusk__1759295781196927438_2
Downloaded image: elonmusk__1759295781196927438_3
Downloaded image: IronPineapple___1760394643265462645_1
D